In [1]:
!pip install sec-api


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from sec_api import QueryApi

SEC_API_KEY = ''

queryApi = QueryApi(api_key=SEC_API_KEY)

def is_public_company(company_name):
    query = {
        "query": { 
            "query_string": { 
                "query": f"companyName:\"{company_name}\" AND formType:\"10-K\""
            } 
        },
        "from": "0",
        "size": "1",
        "sort": [{ "filedAt": { "order": "desc" } }]
    }
    filings = queryApi.get_filings(query)
    if filings and filings.get('filings'):
        return True
    else:
        return False




In [40]:
company_list = ['Apple Inc', 'Acme Corporation']
for company in company_list:
    if is_public_company(company):
        print(f"{company} is a public company.")
    else:
        print(f"{company} is likely a private company.")


Apple Inc is a public company.
Acme Corporation is likely a private company.


In [41]:
import pandas as pd
df_mfm = pd.read_csv('MFM names to match(in).csv')

In [ ]:
public_list = []
private_list = []
for company in df_mfm['nport_name']:
    if is_public_company(company):
        public_list.append(company)
    else:
        private_list.append(company)

In [8]:
public_list

['Fannie Mae',
 'Block, Inc.',
 'Federal National Mortgage Association',
 'Morgan Stanley',
 'MORGAN STANLEY',
 'Prosper Funding LLC']

In [9]:
private_list

['FNMA GTD MTG PASS THRU CTF',
 'Freddie Mac',
 'UGCard',
 'Government National Mortgage Association',
 'Upstart Network, Inc.',
 'United States Treasury',
 'FED HOME LOAN MTG CORP - GOLD',
 'GNMA GUARANTEED PASS THRU CERT',
 'Forward Foreign Currency Contract',
 'Fannie Mae Pool',
 'HSBC Bank PLC',
 'FNMA',
 'GNMA',
 'Freedom Financial Asset Management LLC',
 'Government National Mortgage A',
 'Options Clearing Corp.',
 'U.S. DOLLARS',
 'Federal National Mortgage Association, Inc.',
 'GNMA II GTD PASS THRU CTF MULTIPLE ISSUER',
 'United States of America',
 'Federal Home Loan Mortgage Corporation',
 'United States Treasury Note/Bond']

In [11]:
!pip install sec-edgar



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [26]:
import requests
import time
import json


In [34]:
headers = {
    'User-Agent': 'Ela elahe.paikari@forgeglobal.com'
}


In [35]:
def load_company_tickers():
    url = 'https://www.sec.gov/files/company_tickers.json'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            # Convert the data to a list for easier searching
            company_list = [entry for entry in data.values()]
            return company_list
        except ValueError as e:
            print(f"JSON decoding failed: {e}")
    else:
        print(f"Failed to retrieve company tickers: HTTP {response.status_code}")
    return []



In [29]:
def get_company_cik(company_name, company_list):
    for entry in company_list:
        name = entry['title'].lower()
        if company_name.lower() == name:
            cik_str = str(entry['cik_str']).zfill(10)
            return cik_str
    # If exact match not found, try partial match
    for entry in company_list:
        name = entry['title'].lower()
        if company_name.lower() in name:
            cik_str = str(entry['cik_str']).zfill(10)
            return cik_str
    return None


In [30]:
def has_10k_filings(cik):
    submissions_url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    response = requests.get(submissions_url, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            filings = data.get('filings', {}).get('recent', {})
            forms = filings.get('form', [])
            if '10-K' in forms:
                return True
        except ValueError as e:
            print(f"JSON decoding failed: {e}")
    else:
        print(f"Failed to retrieve filings for CIK {cik}: HTTP {response.status_code}")
    return False


In [ ]:
def is_public_company(company_name, company_list):
    cik = get_company_cik(company_name, company_list)
    if cik:
        print(f"CIK for {company_name}: {cik}")
        time.sleep(0.2)  # Respect SEC rate limits
        if has_10k_filings(cik):
            return True
        else:
            print(f"No 10-K filings found for {company_name}.")
    else:
        print(f"CIK not found for {company_name}.")
    return False


In [42]:
company_list_data = load_company_tickers()

In [ ]:

company_list = ['Apple Inc.', 'Acme Corporation', 'Alphabet Inc.']

for company in company_list:
    print(f"Checking if {company} is a public company...")
    if is_public_company(company, company_list_data):
        print(f"{company} is a public company.")
    else:
        print(f"{company} is likely a private company.")
    print("-" * 50)
    time.sleep(0.2)  # Respect SEC rate limits




Checking if Apple Inc. is a public company...
CIK for Apple Inc.: 0000320193
Apple Inc. is a public company.
--------------------------------------------------
Checking if Acme Corporation is a public company...
CIK not found for Acme Corporation.
Acme Corporation is likely a private company.
--------------------------------------------------
Checking if Alphabet Inc. is a public company...
CIK for Alphabet Inc.: 0001652044
Alphabet Inc. is a public company.
--------------------------------------------------


In [43]:
import pandas as pd
df_mfm = pd.read_csv('MFM names to match(in).csv')

In [ ]:
public_list = []
private_list = []

for company in df_mfm['nport_name']:
    if is_public_company(company, company_list_data):
        public_list.append(company)
    else:
        private_list.append(company)
    time.sleep(0.2)  # Respect SEC rate limits


In [ ]:
len(public_list)
len(private_list)

In [49]:
private_list_df = pd.DataFrame(private_list, columns=['nport_name'])
private_list_df.to_csv('private_companies.csv', index=False)

In [ ]:
# Function to Check for 10-K Filings Using SEC API

def has_10k_filings_sec_api(cik):
    search_url = f"https://data.sec.gov/submissions/CIK{cik.zfill(10)}.json"
    headers = {'User-Agent': 'Your Name your.email@example.com'}
    response = requests.get(search_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        filings = data.get('filings', {}).get('recent', {})
        if 'form' in filings:
            forms = filings['form']
            if '10-K' in forms:
                return True
    else:
        print(f"Failed to retrieve filings for CIK {cik}: {response.status_code}")
    return False